In [216]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re
import random

In [221]:
# Set up headless browser
options = Options()
#options.add_argument("--headless")  # run in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("window-size=1200x800")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# Set up driver (change path to chromedriver if needed)
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    });
    """
})

{'identifier': '2'}

In [198]:
game_links = []
base_browse_url = "https://boardgamegeek.com/browse/boardgame/page/"
for page in range(1, 2):
    browse_url = base_browse_url + str(page)
    driver.get(browse_url)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    all_href = soup.find_all("a", class_="primary")
    for game in all_href:  
        game_links.append("https://boardgamegeek.com" + game["href"])



In [220]:
game_links[0:3]

['https://boardgamegeek.com/boardgame/224517/brass-birmingham',
 'https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1',
 'https://boardgamegeek.com/boardgame/342942/ark-nova']

In [222]:
all_games_data = []
for url in game_links[0:3]:
    game_data = {}
    print("________________________________________")
    print(url)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    title_tag = soup.find("span", itemprop="name")
    if title_tag:
        title = title_tag.get_text(strip=True)
    else:
        title = None
    

    weight_span = soup.find("span", class_=re.compile("gameplay-weight"))
    if weight_span:
        weight = weight_span.get_text(strip=True)
    else:
        weight = None

    panel = soup.find("div", class_="feature-description")
    if panel:
        type_tag = panel.find("a")
        if type_tag:
            game_type = type_tag.get_text(strip=True)
        else:
            game_type = None
    else:
        game_type = None

    rating_span = soup.find("span", itemprop="ratingValue")
    if rating_span:
        rating = rating_span.get_text(strip=True)
    else:
        rating = None

    age_span = soup.find("span", itemprop="suggestedMinAge")
    if age_span:
        min_age = age_span.get_text(strip=True)
    else:
        min_age = None

    num_player_container = soup.find("li", itemprop="numberOfPlayers")
    if num_player_container:
        text_lines = num_player_container.find_all("span", class_="ng-binding ng-scope")
        count = "".join(i.get_text(strip=True) for i in text_lines)
    else:
        count = None

    
    desc_container = soup.find("article", class_="game-description-body ng-scope")
    if desc_container:
        paragraphs = desc_container.find_all("p")
        description = "\n".join(p.get_text(strip=True) for p in paragraphs)
    else:
        description = None
    print(f"{title} | Rating: {rating} | Weight: {weight} | Type: {game_type} | Minimum Age: {min_age} | Player Count: {count}")
    print(description)
    game_data['title'] = title
    game_data['rating'] = rating
    game_data['min_age'] = min_age
    game_data['player_count'] = count
    game_data['type'] = game_type
    game_data['weight'] = weight
    game_data['description'] = description


________________________________________
https://boardgamegeek.com/boardgame/224517/brass-birmingham
Brass: Birmingham | Rating: 8.6 | Weight: 3.87 | Type: Strategy | Minimum Age: 14 | Player Count: 2–4–
Brass: Birminghamis an economic strategy game sequel to Martin Wallace's 2007 masterpiece,Brass.Brass: Birminghamtells the story of competing entrepreneurs in Birmingham during the industrial revolution between the years of 1770 and 1870.
It offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build and establish your industries and network in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.
Each round, players take turns according to the turn order track, receiving two actions to perfor

In [223]:
df = pd.DataFrame(all_games_data)
df.to_csv("boardgames_dataset.csv", index=False)